In [ ]:
from keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense, Activation
from keras.models import Model, Sequential
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
import numpy as np

In [ ]:
# load the dataset but only keep the top n words, zero the rest
top_words = 500
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [ ]:
# truncate and pad input sequences
max_review_length = timesteps = 50
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [ ]:
X_train_encoded = to_categorical(X_train)

In [ ]:
embedding_vector_length = 32
latent_dim = 100

In [ ]:
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(latent_dim))
model.add(RepeatVector(max_review_length))
model.add(LSTM(embedding_vector_length, return_sequences=True))
model.add(TimeDistributed(Dense(top_words)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(X_train, X_train_encoded, batch_size = 100, epochs = 1)

In [ ]:
del X_train_encoded
X_test_encoded = to_categorical(X_test)

In [ ]:
score = model.evaluate(X_test, X_test_encoded, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])
del X_test_encoded

The b) and c) parts follow hereafter -

In [ ]:
from keras import backend as K

get_encoder_output = K.function([model.layers[0].input],
                                  [model.layers[1].output])

In [ ]:
encoded_reviews_train = get_encoder_output([X_train])[0]
encoded_reviews_test = get_encoder_output([X_test])[0]

In [ ]:
model2 = Sequential()
model2.add(Dense(100, input_shape = [latent_dim]))
model2.add(Dense(10))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

In [ ]:
model2.fit(encoded_reviews, y_train, validation_data=(encoded_reviews_test, y_test),epochs=3, batch_size=64)

In [ ]:
score = model2.evaluate(encoded_reviews_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])